# Notebook principal para o pacote `robot_dynamics`

Este notebook age como um `main` executável no Colab (ou localmente), clonando o repositório antes de importar o pacote.
- Ajuste `REPO_URL` para o repositório Git que deseja usar.
- Após o clone, o caminho do repositório é adicionado ao `sys.path`, permitindo edições diretas nos módulos.
- Inclui um exemplo mínimo de cinemática e dinâmica para um robô planar de 2 DOF.


In [ ]:
import os
import subprocess
import sys

# Substitua pelo seu repositório (por exemplo, GitHub/Drive).
REPO_URL = "https://github.com/<usuario>/PFPython.git"
TARGET_DIR = "PFPython"

if not os.path.exists(TARGET_DIR):
    subprocess.run(["git", "clone", REPO_URL, TARGET_DIR], check=True)

repo_path = os.path.abspath(TARGET_DIR)
if repo_path not in sys.path:
    sys.path.insert(0, repo_path)

print(f"Usando repositório em: {repo_path}")


## Importar utilitários do pacote
Caso você edite o código em `robot_dynamics/`, basta reexecutar as células abaixo para refletir as mudanças.


In [ ]:
import sympy as sp

from robot_dynamics import (
    RobotModel,
    build_links_from_data,
    dynamics,
    forward_kinematics,
    parse_axis_order,
    spatial_jacobians,
)


## Exemplo rápido (2 DOF planar)
Os símbolos e parâmetros podem ser ajustados conforme necessário.


In [ ]:

from IPython.display import display, Markdown

# Símbolos
q1, q2 = sp.symbols("q1 q2")
dq1, dq2 = sp.symbols("dq1 dq2")
ddq1, ddq2 = sp.symbols("ddq1 ddq2")
a1, a2 = sp.symbols("a1 a2")
l1, l2 = sp.symbols("l1 l2")
m1, m2 = sp.symbols("m1 m2")
I1x, I1y, I1z, I2x, I2y, I2z = sp.symbols("I1x I1y I1z I2x I2y I2z")
g = sp.symbols("g")

# Dados DH (theta, d, a, alpha) e eixos
dh_params = [
    (q1, 0, a1, 0),
    (q2, 0, a2, 0),
]
joint_types, axes = parse_axis_order(["z", "z"])
masses = [m1, m2]
centers_of_mass = [
    sp.Matrix([l1, 0, 0]),
    sp.Matrix([l2, 0, 0]),
]
inertias = [
    sp.diag(I1x, I1y, I1z),
    sp.diag(I2x, I2y, I2z),
]

links = build_links_from_data(
    [q1, q2],
    joint_types,
    axes,
    dh_params,
    masses,
    centers_of_mass,
    inertias,
)
robot = RobotModel(links=links, gravity=sp.Matrix([0, 0, -g]))

Ts, origins = forward_kinematics(robot)
Jvs, Jws = spatial_jacobians(robot, Ts, origins)

replacements, kinetic_opt, M_opt, C_opt, H_opt, G_opt, tau_opt = dynamics(
    robot,
    [q1, q2],
    [dq1, dq2],
    [ddq1, ddq2],
    parallel=False,
)

sub_map = dict(replacements)

def apply_replacements(expr):
    if hasattr(expr, "shape"):
        return sp.Matrix(expr).xreplace(sub_map)
    return expr.xreplace(sub_map)

replacement_lines = [
    f"- ${sp.latex(sym)} = {sp.latex(expr)}$" for sym, expr in replacements
]

display(Markdown("### Subexpressões extraídas (x_i)"))
if replacement_lines:
    display(Markdown("\n".join(replacement_lines)))
else:
    display(Markdown("- Nenhuma subexpressão extraída."))

T_ee = Ts[-1]
sections = [
    ("Cinemática direta (T_ee)", T_ee),
    ("Jacobiano linear Jv (elo final)", Jvs[-1]),
    ("Jacobiano angular Jw (elo final)", Jws[-1]),
    ("Energia cinética total", kinetic_opt),
    ("Matriz de inércia M", M_opt),
    ("Matriz de Coriolis/Centrífuga C", C_opt),
    ("Matriz H = C + G", H_opt),
    ("Vetor de gravidade G", G_opt),
    ("Torques τ", tau_opt),
]

for title, value in sections:
    display(Markdown(f"### {title}"))
    display(apply_replacements(value))
